In [3]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization,Conv2D,MaxPooling2D
import os
from keras.preprocessing.image import ImageDataGenerator

N_class=5  #no. of class i.e no. of emotion
img_dimension=48 # 48x48 image
batch_size=32  # for working in cpu.

validationdata = (r'D:\AI Project\validation') 
traindata = (r'D:\AI Project\train')

training_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.4,
    height_shift_range=0.4,
    zoom_range=0.3,
    shear_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)
validation_datagen=ImageDataGenerator(rescale=1./255)

training_datagenrator=training_datagen.flow_from_directory(
    traindata,
    color_mode='grayscale',
    target_size=(img_dimension,img_dimension),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_datagenrator=training_datagen.flow_from_directory(
    traindata,
    color_mode='grayscale',
    target_size=(img_dimension,img_dimension),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)
model = Sequential()

# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_dimension,img_dimension,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_dimension,img_dimension,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(N_class,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('Emotion_little_vgg.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25

history=model.fit_generator(
                training_datagenrator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_datagenrator,
                validation_steps=nb_validation_samples//batch_size)

Found 24256 images belonging to 5 classes.
Found 24256 images belonging to 5 classes.
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
activation_23 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_24 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 48, 48, 32)        128       
__________________________________

755/755 [==============================] - 344s 455ms/step - loss: 1.1632 - accuracy: 0.5294 - val_loss: 0.9391 - val_accuracy: 0.5625

Epoch 00010: val_loss improved from 1.02583 to 0.93915, saving model to Emotion_little_vgg.h5
Epoch 11/25
755/755 [==============================] - 356s 471ms/step - loss: 1.1461 - accuracy: 0.5377 - val_loss: 1.1526 - val_accuracy: 0.5470

Epoch 00011: val_loss did not improve from 0.93915
Epoch 12/25
755/755 [==============================] - 383s 508ms/step - loss: 1.1211 - accuracy: 0.5476 - val_loss: 0.8629 - val_accuracy: 0.5635

Epoch 00012: val_loss improved from 0.93915 to 0.86289, saving model to Emotion_little_vgg.h5
Epoch 13/25
755/755 [==============================] - 367s 486ms/step - loss: 1.1006 - accuracy: 0.5629 - val_loss: 1.0502 - val_accuracy: 0.5558

Epoch 00013: val_loss did not improve from 0.86289
Epoch 14/25
755/755 [==============================] - 390s 516ms/step - loss: 1.0871 - accuracy: 0.5712 - val_loss: 0.9845 - val_